# Registry Pattern for Dependency Injection

### Service Locator:
The registry can act as a service locator, where dependencies are registered and then injected into classes that need them. This is sometimes called the "service locator" pattern, which is a variation of the registry pattern.
### Dependency Registration:
When setting up a DI container, you can use a registry to register dependencies. This allows you to map interfaces or abstract classes to their concrete implementations.
### Runtime Resolution:
The registry can be used to resolve dependencies at runtime, allowing for more dynamic dependency injection scenarios.

In [2]:
# Service Locator
class Registry:
    def __init__(self):
        self._services = {}

    def register(self, name, service):
        self._services[name] = service

    def get(self, name):
        return self._services.get(name)

# Dependency
class DatabaseService:
    def connect(self):
        print("Connecting to database")

# Client
class UserRepository:
    def __init__(self, db_service):
        self.db_service = db_service

    def save_user(self):
        self.db_service.connect()
        print("Saving user")

# Set up the registry
registry = Registry()
registry.register("db_service", DatabaseService())

# Inject the dependency
db_service = registry.get("db_service")
user_repo = UserRepository(db_service)

# Use the injected dependency
user_repo.save_user()

Connecting to database
Saving user


In [3]:
# Dependency Registration
# Register services by interface name (in this example)
# Singleton vs transient (prototye)
# Factory functions

class Registry:
    def __init__(self):
        self._services = {}

    def register(self, interface, implementation):
        self._services[interface] = implementation

    def get(self, interface):
        return self._services.get(interface)

# Interfaces
class ILogger:
    def log(self, message): pass

class IDatabase:
    def query(self, sql): pass

# Implementations
class ConsoleLogger(ILogger):
    def log(self, message):
        print(f"Log: {message}")

class SQLiteDatabase(IDatabase):
    def query(self, sql):
        print(f"Executing query: {sql}")

# Registration
registry = Registry()
registry.register(ILogger, ConsoleLogger())
registry.register(IDatabase, SQLiteDatabase())

# Usage
logger = registry.get(ILogger)
db = registry.get(IDatabase)

logger.log("Application started")
db.query("SELECT * FROM users")

Log: Application started
Executing query: SELECT * FROM users


In [4]:
# Runtime Resolution
# lazy loading of the serivce
# WARNING: runtime resolution hides the dependency from public interface or method signatures, 
#          reducing the readability of the code and obscured initialization requirements.

class UserService:
    def __init__(self, registry):
        self.registry = registry

    def create_user(self, username):
        logger = self.registry.get(ILogger)
        db = self.registry.get(IDatabase)

        logger.log(f"Creating user: {username}")
        db.query(f"INSERT INTO users (username) VALUES ('{username}')")
        logger.log("User created successfully")

# Using the previously defined Registry and registered dependencies
registry = Registry()
registry.register(ILogger, ConsoleLogger())
registry.register(IDatabase, SQLiteDatabase())

# Create UserService with registry
user_service = UserService(registry)

# Use the service, which will resolve dependencies at runtime
user_service.create_user("john_doe")

Log: Creating user: john_doe
Executing query: INSERT INTO users (username) VALUES ('john_doe')
Log: User created successfully


In [1]:
# Registry factory function
# This solves the hidden dependency problem in the example above.

from typing import Callable, Any, Dict
from abc import ABC, abstractmethod

class DIContainer:
    def __init__(self):
        self._services: Dict[str, Callable[[], Any]] = {}

    def register(self, name: str, factory: Callable[[], Any]):
        self._services[name] = factory

    def resolve(self, name: str) -> Any:
        if name not in self._services:
            raise KeyError(f"Service '{name}' not registered.")
        return self._services[name]()

class ILogger(ABC):
    @abstractmethod
    def log(self, message: str):
        pass

class ConsoleLogger(ILogger):
    def log(self, message: str):
        print(f"[LOG] {message}")

class DataService:
    def __init__(self, logger: ILogger):
        self.logger = logger

    def process_data(self):
        self.logger.log("Processing data...")

# Usage example
container = DIContainer()

# Register services with factory functions
container.register("logger", lambda: ConsoleLogger())
container.register("data_service", lambda: DataService(container.resolve("logger")))

# Resolve and use services
service1 = container.resolve("data_service")
service1.process_data()

service2 = container.resolve("data_service")
service2.process_data()

print(f"Are services the same instance? {service1 is service2}")

[LOG] Processing data...
[LOG] Processing data...
Are services the same instance? False
